# Analyzing CIA Factbook Data Using SQL

This project will involve analysing the [CIA Factbook Data](https://www.cia.gov/library/publications/the-world-factbook/) containing information regarding the population and area of each world country/territory. 

Some of the questions we will attempt to answer:

- Which countries are the most densely populated?
- Which countries are growing the most?

## Connecting to Database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Data Overview

In [6]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [7]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Here are the descriptions for some of the columns:

name - The name of the country.

area - The total land and sea area of the country.

population - The country's population.

population_growth- The country's population growth as a percentage.

birth_rate - The country's birth rate, or the number of births a year per 1,000 people.

death_rate - The country's death rate, or the number of death a year per 1,000 people.

area- The country's total area (both land and water).

area_land - The country's land area in square kilometers.

area_water - The country's waterarea in square kilometers.

-----------------------------------------------------------------------------

Let's start by calculating some summary statistics and see what they tell us.

In [8]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


There seem to be some outliers, as a minimum population of 0 and a maximum of over 7 billion would seem to suggest.

In [12]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MIN(population)
FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [13]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MAX(population)
FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Our results show that Antarctica is responsible for the 0 population (which makes sense) whilst the 7 billion is the total global population. It makes sense to include Antarctica in any analysis and exclude the World values.

In [15]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


## Population Density

In [23]:
%%sql 
SELECT ROUND(AVG(population), 0) AS Average_Population, ROUND(AVG(area), 2) AS Average_Area
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


Average_Population,Average_Area
32242667.0,555093.55


Selecting countries with above average population and below average area (i.e. some of the more densely populated countries)

In [27]:
%%sql 
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts)
AND area < (SELECT AVG(area)
            FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


Let's calculate the 10 most densely populated countries.

In [37]:
%%sql
SELECT name, population, area,ROUND((CAST(population AS FLOAT)/CAST(area AS FLOAT)), 1) AS Population_Density
FROM facts
WHERE name != 'World'
ORDER BY Population_Density DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population,area,Population_Density
Macau,592731,28,21169.0
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.3
Hong Kong,7141106,1108,6445.0
Gaza Strip,1869055,360,5191.8
Gibraltar,29258,6,4876.3
Bahrain,1346613,760,1771.9
Maldives,393253,298,1319.6
Malta,413965,316,1310.0
Bermuda,70196,54,1299.9


The countries heading this list are generally much smaller countries (by area and population) than we selected previously. We could also do this more specifically by land area rather than just simply area.

In [39]:
%%sql
SELECT name, population, area_land,ROUND((CAST(population AS FLOAT)/CAST(area_land AS FLOAT)), 1) AS Population_Density
FROM facts
WHERE name != 'World'
ORDER BY Population_Density DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population,area_land,Population_Density
Macau,592731,28,21169.0
Monaco,30535,2,15267.5
Singapore,5674472,687,8259.8
Hong Kong,7141106,1073,6655.3
Gaza Strip,1869055,360,5191.8
Gibraltar,29258,6,4876.3
Bahrain,1346613,760,1771.9
Maldives,393253,298,1319.6
Malta,413965,316,1310.0
Bermuda,70196,54,1299.9


Our list hasn't changed much (in fact the composition and order of the top 10 is identical) but still gives us a slightly more specific measure of Population Density.

## Which Countries will have the highest population increase

We can figure out which countries will have the highest number of people added to their population in the year 2016.

In [43]:
%%sql
SELECT name, population, population_growth, CAST(population*((population_growth/100)) AS INT) AS added_people
FROM facts
WHERE name != 'World'
ORDER BY added_people DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population,population_growth,added_people
India,1251695584,1.22,15270686
China,1367485388,0.45,6153684
Nigeria,181562056,2.45,4448270
Pakistan,199085847,1.46,2906653
Ethiopia,99465819,2.89,2874562
Bangladesh,168957745,1.6,2703323
United States,321368864,0.78,2506677
Indonesia,255993674,0.92,2355141
"Congo, Democratic Republic of the",79375136,2.45,1944690
Philippines,100998376,1.61,1626073


Unsurprisingly, the two countries with the highest populations head this list as well, although the difference between the population growth rates of India and China is stark. This is most likely a result of the famous one child policy, which was in place between 1980 and 2016. (https://www.britannica.com/topic/one-child-policy)

Lets also calculate the 'Rate of Natural Population Increase', a measure of the difference between the birth rate and the death rate.

In [45]:
%%sql 
SELECT name, ROUND(birth_rate - death_rate, 2) AS natural_pop_increase
FROM facts
WHERE natural_pop_increase IS NOT NULL
ORDER BY natural_pop_increase DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,natural_pop_increase
Malawi,33.15
Uganda,33.1
Niger,33.03
Burundi,32.74
Mali,32.1
Burkina Faso,30.31
Zambia,29.46
Ethiopia,29.08
South Sudan,28.73
Tanzania,28.39


This measurement returns a very different list of countries (as it takes no account of migration rates etc.). It is a list dominated by African countries (which we expect to have lower GDP/capita). The cause is likely too be the less readily available birth control compared to the wealthier countries. 

Let's have a look at the other end of the spectrum.

In [46]:
%%sql 
SELECT name, ROUND(birth_rate - death_rate, 2) AS natural_pop_increase
FROM facts
WHERE natural_pop_increase IS NOT NULL
ORDER BY natural_pop_increase
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,natural_pop_increase
Bulgaria,-5.52
Serbia,-4.58
Latvia,-4.31
Lithuania,-4.17
Ukraine,-3.74
Hungary,-3.57
Germany,-2.95
Slovenia,-2.95
Romania,-2.76
Croatia,-2.73


The countries with the lowest 'Rate of Natural Population Increase' seem to be dominated by Eastern European countries. The reason for this precise localisation is difficult to pinpoint, although the increasing number of women that are now choosing to pursue careers into their 30s and beyond may have an underlying factor.